<a href="https://colab.research.google.com/github/tombartos/tp-information-retrieval-with-llm-student-version/blob/main/1-Recherche%20d'information%20classique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Partie 1. - Recherche d'Information classique

Dans cette partie, nous allons mettre en œuvre des principes et modèles classiques de Recherche d'Information. Le jeu de données est une collection de livres au format texte (.txt) de Henry Rider Haggard. Jetez un œil à ces documents dans le dossier _data_.

En sortie de ce module, vous serez capable de :

- Construire un index inversé ;
- Effectuer des requêtes simples selon le modèle booléen :
- Calculer la pondération des termes selon la méthode TF-IDF ;
- Mettre en œuvre le modèle vectoriel de recherche d'information et y appliquer des requêtes.

### Import des bibliothèques logicielles et configuration

Les lignes suivantes permettent d'instancier un objet de la classe `IRSystem` représentant notre moteur de recherche et de charger les données en RAM.

In [36]:
import os

# Vérifie si le code est exécuté sur Google Colab
if 'COLAB_GPU' in os.environ:
    # Commandes à exécuter uniquement sur Google Colab
    !git clone https://github.com/vincentmartin/tp-information-retrieval-with-llm-student-version.git
    %cd tp-information-retrieval-with-llm-student-version
else:
    # Commandes à exécuter si ce n'est pas sur Google Colab
    print("Pas sur Google Colab, ces commandes ne seront pas exécutées.")

Cloning into 'tp-information-retrieval-with-llm-student-version'...
remote: Enumerating objects: 2254, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 2254 (delta 24), reused 33 (delta 17), pack-reused 2212 (from 1)
Receiving objects: 100% (2254/2254), 13.92 MiB | 15.78 MiB/s, done.
Resolving deltas: 100% (26/26), done.
/content/tp-information-retrieval-with-llm-student-version/tp-information-retrieval-with-llm-student-version/tp-information-retrieval-with-llm-student-version/tp-information-retrieval-with-llm-student-version


#### Chargement des données

Les lignes ci-dessous permettent de charger les données qui sont un ensemble de 60 livres au format texte (.txt) d'[Henry Rider Haggard](https://fr.wikipedia.org/wiki/Henry_Rider_Haggard).


In [37]:
from classic_ir.IRSystem import *

# !rm -rf ./data/RiderHaggard/stemmed
ir_system = IRSystem()
ir_system.read_data('./data/RiderHaggard') # chargement des données et prétraitement des documents (stemming).

Reading in documents...
Stemming Documents...
Allan Quatermain 711.txt
    Doc 1 of 60: Allan Quatermain
Jess 5898.txt
    Doc 2 of 60: Jess
Ayesha, the Return of She 5228.txt
    Doc 3 of 60: Ayesha, the Return of She
Allan and the Ice Gods (1927) 0200201.txt
    Doc 4 of 60: Allan and the Ice Gods (1927)
Lysbeth, a Tale of the Dutch 5754.txt
    Doc 5 of 60: Lysbeth, a Tale of the Dutch
The People of the Mist 6769.txt
    Doc 6 of 60: The People of the Mist
Queen of the Dawn (1925) 0200381.txt
    Doc 7 of 60: Queen of the Dawn (1925)
Long Odds 1918.txt
    Doc 8 of 60: Long Odds
The Mahatma and the Hare 2764.txt
    Doc 9 of 60: The Mahatma and the Hare
Dawn 10892.txt
    Doc 10 of 60: Dawn
Marie An Episode in The Life of the late Allan Quatermain 1690.txt
    Doc 11 of 60: Marie An Episode in The Life of the late Allan Quatermain
Child of Storm 1711.txt
    Doc 12 of 60: Child of Storm
Stella Fregelius 6051.txt
    Doc 13 of 60: Stella Fregelius
Red Eve 3094.txt
    Doc 14 of 60: R

### Exercice 1. - Construction de l'index inversé

Ce premier exercice a pour objectif de construire l'index inversé non positionnel. L'attribut `self.inverted_index` est un tableau associatif qui associe une liste d'entiers (docIDs) à un mot (word).

Documentation ici https://docs.python.org/3/library/collections.html#collections.defaultdict.

Exercice : modifier la fonction `index` pour calculer l'index inversé.

Le résultat ci-dessous indique que vous avez réussi.
```
===== Running tests =====
Inverted Index Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [38]:
# Exercice 1. Indexation

def index(self):
    """
    Construit l'index inversé et le stocke dans self.inverted_index.

    Dans le code ci-dessous, seul le dictionnaire des tokens est construit. Les postings lists sont vides.
    """
    print("Indexing...")
    self.tf = defaultdict(Counter) # tf est un dictionnaire qui à un document associe un Counter de mots.
    inverted_index = defaultdict(list) # inverted_index est un dictionnaire qui à un mot associe une liste de documents.
    for word in self.vocab:
        inverted_index[word] = list()
    i=0
    for doc in self.docs:
      for word in doc:
        if i not in inverted_index[word]:
          inverted_index[word].append(i)
      i+=1



    self.inverted_index = inverted_index

# Ne pas modifier les lignes ci-dessous
IRSystem.index = index
ir_system.index()
run_tests(ir_system, part=0)

Indexing...
===== Running tests =====
Inverted Index Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000


### Exercice 2. - Recherche booléenne

Ce deuxième exercice a pour objectif d'implémenter la recherche booléenne. La requête `query` est une liste de mots _lemmatisés_ et l'algorithme doit rechercher les documents qui contiennent TOUS ces mots.


Exercice : modifier la fonction `boolean_retrieve` pour implémenter la recherche booléenne.


Le résultat ci-dessous indique que vous avez réussi.
```
===== Running tests =====
Boolean Retrieval Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [39]:
# Exercice 2. Recherche booléenne
def boolean_retrieve(self, query):
    """
    A partir d'une requête sous la forme d'une liste de mots *lemmatisés*,
    retourne la liste des documents dans lesquels *tous* ces mots apparaissent (ie une requête AND).
    Retourne une liste vide si la requête ne retourne aucun document.

    Dans le code ci-dessous, tous les documents répondent.
    """
    docs = list()
    for docid in self.inverted_index[query[0]]:
      present = True
      for qword in query[1:]:
        if docid not in self.inverted_index[qword]:
          present = False
          break     # On sort de la boucle si le doc qu'on cherche n'est pas présent dans la liste et on passe au doc suivant
      if present:
        docs.append(docid)

    return docs

# Ne pas modifier les lignes ci-dessous
IRSystem.boolean_retrieve = boolean_retrieve
run_tests(ir_system, part=1)

===== Running tests =====
Boolean Retrieval Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000


### Exercice 3. - Calcul des poids TF-IDF des termes dans les documents

Dans ce troisième exercice, l'objectif est de pré-calculer les poids TF-IDF pour chaque terme dans chaque document. Pour ce faire, appliquer la formule vue en cours. Utiliser le logarithme en base 10.


Exercice : modifier la fonction `boolean_retrieve` pour implémenter la recherche booléenne.

Le résultat ci-dessous indique que vous avez réussi.
```
Calculating tf-idf...
===== Running tests =====
TF-IDF Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [54]:
def run_tests(irsys, part=None):
    print ("===== Running tests =====")

    ff = open('./data/queries.txt')
    questions = [xx.strip() for xx in ff.readlines()]
    ff.close()
    ff = open('./data/solutions.txt')
    solutions = [xx.strip() for xx in ff.readlines()]
    ff.close()

    epsilon = 1e-4
    #for part in range(4):
    points = 0
    num_correct = 0
    num_total = 0

    prob = questions[part]
    soln = json.loads(solutions[part])

    # inverted index test
    if part == 0:
        print ("Inverted Index Test")
        words = prob.split(", ")
        for i, word in enumerate(words):
            num_total += 1
            posting = irsys.get_posting_unstemmed(word)
            if set(posting) == set(soln[i]):
                num_correct += 1

    # boolean retrieval test
    elif part == 1:
        print ("Boolean Retrieval Test")
        queries = prob.split(", ")
        for i, query in enumerate(queries):
            num_total += 1
            guess = irsys.query_retrieve(query)
            if set(guess) == set(soln[i]):
                num_correct += 1

    # tfidf test
    elif part == 2:
        print ("TF-IDF Test avec print")
        queries = prob.split("; ")
        queries = [xx.split(", ") for xx in queries]
        queries = [(xx[0], int(xx[1])) for xx in queries]
        for i, (word, doc) in enumerate(queries):
            num_total += 1
            guess = irsys.get_tfidf_unstemmed(word, doc)
            print(f"guess: {guess}, sol: {soln[i]}")
            if guess >= float(soln[i]) - epsilon and \
                    guess <= float(soln[i]) + epsilon:
                num_correct += 1

    # cosine similarity test
    elif part == 3:
        print ("Cosine Similarity Test")
        queries = prob.split(", ")
        for i, query in enumerate(queries):
            num_total += 1
            ranked = irsys.query_rank(query)
            top_rank = ranked[0]
            print(f"top0: {top_rank[0]}, top1: {top_rank[1]}, guess0: {soln[i][0]}, guess1: {soln[i][1]} ")
            if top_rank[0] == soln[i][0]:
                if top_rank[1] >= float(soln[i][1]) - epsilon and \
                        top_rank[1] <= float(soln[i][1]) + epsilon:
                    num_correct += 1

    feedback = "%d/%d Correct. Accuracy: %f" % \
            (num_correct, num_total, float(num_correct)/num_total)
    if num_correct == num_total:
        points = 3
    elif num_correct > 0.75 * num_total:
        points = 2
    elif num_correct > 0:
        points = 1
    else:
        points = 0

    print ("    Score: %d Feedback: %s" % (points, feedback))

In [48]:
# Exercice 3. calcul des scores tf-idf
from math import log
def compute_tfidf(self):
    """
    Calcule les scores tf-idf pour tous les mots de tous les documents et les stocke dans self.tfidf.

    Dans le code ci-dessous, les scores tf-idf sont tous nuls.
    """
    print("Calculating tf-idf...")
    self.tfidf = defaultdict(Counter)
    N = len(self.docs)  # N = nombre de documents

    occurs = []
    for i in range(N):
      occurs.append(Counter(self.docs[i]))
    for word in self.vocab:
        idftmp = log(N/len(self.inverted_index[word]), 10)
        for i in range(N):
            try:
                self.tfidf[i][word] = (1+log(occurs[i][word], 10)) * idftmp
            except ValueError:
                self.tfidf[i][word] = 0.
# Ne pas modifier les lignes ci-dessous
IRSystem.compute_tfidf = compute_tfidf
ir_system.compute_tfidf()
run_tests(ir_system, part=2)

Calculating tf-idf...
===== Running tests =====
TF-IDF Test avec print
guess: 0.1333773650324118, sol: 0.13337736503241182
guess: 0.5228787452803376, sol: 0.5228787452803376
guess: 0.23408320603336794, sol: 0.23408320603336796
guess: 0.3916490539534377, sol: 0.39164905395343774
guess: 0.12550382549455122, sol: 0.12550382549455125
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000


### Exercice 4. - Calcul de la similarité cosinus.

Dans ce troisième exercice, l'objectif est de calculer la similarité cosinus entre le vecteur requête `query` et chaque vecteur document. Pour ce faire, appliquer la formule vue en cours en considérant les éléments suivants :
- Ne considérer que la mesure TF pour calculer le poids des termes de la requête (on n'utilise pas IDF).
- Utiliser le logarithme en base 10.

Exercice : modifier la fonction `rank_retrieve` pour implémenter la recherche booléenne.

Le résultat ci-dessous indique que vous avez réussi.
```
===== Running tests =====
Cosine Similarity Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [59]:
# Exercice 4. Similarité cosinus
def rank_retrieve(self, query):
    """
    A partir d'une requête (une liste de mots), retourne une liste de documents (classés par docID) et de scores pour la requête en appliquant la simalirité cosinus.

    Dans l'exemple ci-dessous. C'est la mesure de Jaccard qui est utilisée.
    """
    scores = [0.0 for _ in range(len(self.docs))]

    query_set = set(query)

    for d in range(len(self.docs)):
        doc_set = set(self.docs[d])
        intersection = len(query_set & doc_set)
        #union = len(query_set | doc_set)
        # Calcul de la similarité Jaccard
        #scores[d] = intersection / union if union != 0 else 0.0
        sum1 = 0
        if intersection != 0:
          for word in query_set:
            sum1+= self.tfidf[d][word]
        sum2 = 0
        for tfidf in self.tfidf[d].values():
          sum2+=tfidf**2
        scores[d] = sum1 / (sum2**0.5)



    # Tri des scores
    ranking = [idx for idx, sim in sorted(enumerate(scores),
                                        key=lambda pair: pair[1], reverse=True)]
    results = []
    for i in range(10):
        results.append((ranking[i], scores[ranking[i]]))
    return results

# Ne pas modifier les lignes ci-dessous
IRSystem.rank_retrieve = rank_retrieve
run_tests(ir_system, part=3)

===== Running tests =====
Cosine Similarity Test
top0: 56, top1: 0.010676611271744126, guess0: 56, guess1: 0.010676611271744128 
top0: 59, top1: 0.05041279333939143, guess0: 59, guess1: 0.05041279333939147 
top0: 15, top1: 0.0340578944755209, guess0: 15, guess1: 0.03405789447552096 
top0: 24, top1: 0.015298081869538096, guess0: 24, guess1: 0.015298081869538085 
top0: 21, top1: 0.03518293691585286, guess0: 21, guess1: 0.035182936915852864 
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
